In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
transform = transforms.Compose(
        [transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 500

trainset_big = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainset = torch.utils.data.Subset(trainset_big,list(range(10000)))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

val_set = torch.utils.data.Subset(trainset_big,list(range(10000, 20000)))
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:10<00:00, 15798793.09it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_model = model
    best_loss = 100
    for epoch in range(num_epochs):
        tr_correct = 0
        tr_total = 0
        val_correct = 0
        val_total = 0
        for batch_nr, (data, labels) in enumerate(train_loader):
            
            print("Epoch: ",epoch,"Batch: ",batch_nr)
            # calculate prediction according to our model
            data = data.cuda()
            labels = labels.cuda()
            prediction = model.forward(data)
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)
            
            # Backpropagate the loss through the network to find the gradients of all parameters
            loss.backward()
            
            # Update the parameters along their gradients
            optimizer.step()
            
            # Clear stored gradient values
            optimizer.zero_grad()

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    tr_correct+=1
                tr_total +=1



        for batch_nr, (data, labels) in enumerate(val_loader):
            data = data.cuda()  
            labels = labels.cuda()  
            prediction = model.forward(data)
            
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)

            if(loss < best_loss):
                best_loss = loss
                best_model = model

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    val_correct+=1
                val_total +=1

    # print accuracy
    tr_accuracy = tr_correct/tr_total
    val_accuracy = val_correct/val_total
    print(f'Training accuracy:   {str(100*tr_accuracy)[:4]}%.')
    print(f'Validation accuracy: {str(100*val_accuracy)[:4]}%.')

    return best_model

In [5]:
def test_model(model, test_loader):
    val_correct = 0
    val_total = 0
    for batch_nr, (data, labels) in enumerate(test_loader):

        data = data.cuda()  
        labels = labels.cuda()  
        prediction = model.forward(data)
        
        # calculate accuracy
        for i in range(len(data)):    
            guess = torch.argmax(prediction[i])
            if(guess.item() == labels[i]):
                val_correct+=1
            val_total +=1

    # primt accuracy
    val_accuracy = val_correct/val_total
    print(f'Test accuracy: {str(100*val_accuracy)[:4]}%.')

In [6]:
alexNet_fineTuning = torchvision.models.alexnet(pretrained=True)
alexNet_fineTuning.classifier[6] = nn.Linear(4096,10)

# alexNet_fineTuning.eval

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:00<00:00, 349MB/s]


In [7]:
# Define for the optimization algorithm which parameters we want to update during training

# Define our loss function
criterion = torch.nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(alexNet_fineTuning.parameters(), lr=0.001)

In [8]:
alexNet_fineTuning = alexNet_fineTuning.cuda()

In [9]:

# Run the training step
train_model(alexNet_fineTuning,criterion,optimizer,trainloader,val_loader,num_epochs=10)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
# Define for the optimization algorithm which parameters we want to update during training
alexNet_featureExtraction = torchvision.models.alexnet(pretrained=True)

for param in alexNet_featureExtraction.parameters():
    param.requires_grad = False

alexNet_featureExtraction.classifier[6] = nn.Linear(4096,10)
# find the paramaters we want to update during training
params_to_update = []
for param in alexNet_featureExtraction.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

for name, param in alexNet_featureExtraction.named_parameters():
    if param.requires_grad:
        print (name, param.data)

# Define our loss function
criterion_fe = torch.nn.CrossEntropyLoss()

# Define our optimizer
optimizer_fe = torch.optim.Adam(params_to_update, lr=0.001)

classifier.6.weight tensor([[-0.0015, -0.0141,  0.0024,  ..., -0.0090, -0.0087, -0.0045],
        [-0.0132,  0.0118,  0.0109,  ...,  0.0002,  0.0084, -0.0052],
        [-0.0116, -0.0124,  0.0065,  ...,  0.0053,  0.0077,  0.0012],
        ...,
        [-0.0058, -0.0061, -0.0121,  ...,  0.0107,  0.0145, -0.0037],
        [-0.0084, -0.0150,  0.0088,  ...,  0.0136, -0.0077,  0.0057],
        [ 0.0032,  0.0083, -0.0051,  ..., -0.0114, -0.0068,  0.0012]])
classifier.6.bias tensor([ 0.0072, -0.0004,  0.0048,  0.0009, -0.0081, -0.0046,  0.0025, -0.0065,
         0.0095,  0.0036])


In [11]:
alexNet_featureExtraction = alexNet_featureExtraction.cuda()

In [12]:
# Run the training step
train_model(alexNet_featureExtraction,criterion_fe,optimizer_fe,trainloader,val_loader,num_epochs=10)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [13]:

print("Fine Tuning:")
test_model(alexNet_fineTuning, testloader)

print("\nFeature Extraction:")
test_model(alexNet_featureExtraction, testloader)

Fine Tuning:
Test accuracy: 62.1%.

Feature Extraction:
Test accuracy: 71.2%.


In [14]:
transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))])

batch_size = 500

trainset_MNIST_big = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

trainset_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000)))

trainloader_MNIST = torch.utils.data.DataLoader(trainset_MNIST, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

val_set_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000, 20000)))
val_loader_MNIST = torch.utils.data.DataLoader(val_set_MNIST, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
                                          
testset_MNIST = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader_MNIST = torch.utils.data.DataLoader(testset_MNIST, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

100%|██████████| 9912422/9912422 [00:00<00:00, 79593741.43it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 96986143.97it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20510430.05it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4604913.89it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [15]:
# Chose ResNet-18 as CNN model
resnet = torchvision.models.resnet18(pretrained= False)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# optimizer = torch.optim.Adam(params_to_update, lr=0.001)
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [16]:
resnet=resnet.cuda()

In [17]:
# Trained the model on MNIST data
trained_resnet = train_model(resnet, criterion, optimizer, trainloader_MNIST, val_loader_MNIST, num_epochs=10)
test_model(trained_resnet, testloader_MNIST)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


In [18]:
transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 500

all_svhn_train_data = torchvision.datasets.SVHN(root='./data', split= 'train',
                                        download=True, transform=transform)
all_svhn_test_data = torchvision.datasets.SVHN(root='./data', split= 'test',
                                        download=True, transform=transform)
svhn_train_set = torch.utils.data.Subset(all_svhn_train_data,list(range(10000)))
svhn_val_set  = torch.utils.data.Subset(all_svhn_train_data,list(range(10000, 20000)))
svhn_test_set  = torch.utils.data.Subset(all_svhn_test_data,list(range(10000)))

svhn_train_loader = torch.utils.data.DataLoader(svhn_train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
svhn_val_loader = torch.utils.data.DataLoader(svhn_val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
svhn_test_loader = torch.utils.data.DataLoader(svhn_test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

100%|██████████| 182040794/182040794 [00:11<00:00, 15210779.74it/s]


100%|██████████| 64275384/64275384 [00:06<00:00, 10587969.99it/s]


In [19]:
resnet.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
trained_resnet = trained_resnet.cuda()
print("MNIST trained Resnet on SVHN dataset:")
test_model(trained_resnet, svhn_test_loader)

MNIST trained Resnet on SVHN dataset:
Test accuracy: 10.4%.


In [20]:
trained_resnet_fe = trained_resnet 

# FREEZE all old params
for param in trained_resnet_fe.parameters():
    param.requires_grad = False

# new layer
num_ftrs = trained_resnet_fe.fc.in_features
trained_resnet_fe.fc = nn.Linear(num_ftrs, 10)

# find the paramaters we want to update during training
params_to_update = []
for param in trained_resnet_fe.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

# Define our loss function
criterion_fe = torch.nn.CrossEntropyLoss()

# Define our optimizer
optimizer_fe = torch.optim.Adam(params_to_update, lr=0.001)

In [21]:
trained_resnet_fe=trained_resnet_fe.cuda()

In [22]:
# Run the training step
train_model(trained_resnet_fe, criterion_fe, optimizer_fe, svhn_train_loader, svhn_val_loader, num_epochs=10)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
print("Feature Extraction of MNIST-trained Resnet on SVHN dataset:")
test_model(trained_resnet_fe, svhn_test_loader)

Feature Extraction of MNIST-trained Resnet on SVHN dataset:
Test accuracy: 25.8%.
